In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# Prompte Template을 사용하는 방법
# --------------------------
# t = PromptTemplate.from_template("What is the capital of {country}")

# t.format(country="Turkey")
# prompt template을 사용하는 것의 장점은 유효성 검사를 할 수 있기 때문이다. 예를들어서 여기에서 format에 값을 입력하지 않을 경우 에러가 발생한다.

# -------------------------- 아래와 같이 사용할 수도 있다. 동일한 결과값을 갖는다.
# t = PromptTemplate(
#     template = "What is the capital of {country}",
#     input_variables=["country"],
#     )

# t.format(country="Turkey")
# --------------------------

# Prompt로 전달하는 것보다 예제를 주는 것이 답변을 받을 때 더 좋은 방법이다. (예를들어 답변에 콤마를 찍어서 분리해줘, 소문자로 작성해줘 등등)
# 이 것을 해주는 것이 FewShotPromptTemplate이다.

# 이렇게 답변해줬으면 좋겠어 라는 예제 
examples = [
    {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    },
]

### 1. 예제 없이 실행했을 때, 줄글로 답변을 준다.
# chat.predict("what do you know about France?")
# 결과값 : France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. France is also known for its wine production, fashion industry, and art scene. The country has a diverse landscape, including mountains, beaches, and countryside. French is the official language, and the currency is the Euro. France is a member of the European Union and is one of the most visited countries in the world.

### 2. 답변 예제를 전달해서 예제 형식에 맞게 답변을 받는다. 
# 데이터베이스 등에서 꺼내올 수 있기 때문에 형식화 해주는 과정임. 생략해도 된다.
# 여기의 형식은 위의 examples의 형식과 동일해야한다. - 여기에서는 question과 answer
example_template = """
    Human:{question}
    AI: {answer}
"""

# example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{}") # 위의 형식을 사용하지 않을 경우 이렇게 간단하게 사용할 수도 있음
example_prompt = PromptTemplate.from_template(example_template)

# 예제 리스트들을 이 prompt를 사용하여 형식화 한다.
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?", # 마지막에 추가할 질문
    input_variables=["country"]
)

# 예제 리스트들을 만들어서 fewShotPromptTemplate에 전달한다.
# prompt.format(country="Germany")
# 결과값
# 'Human:What do you know about France?\n    AI: \n    Here is what I know:\n    Capital: Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n    \n\n\n\n    
# Human:What do you know about Italy?\n    AI: \n    I know this:\n    Capital: Rome\n    Language: Italian\n    Food: Pizza and Pasta\n    Currency: Euro\n    \n\n\n\n    
# Human:What do you know about Greece?\n    AI: \n    I know this:\n    Capital: Athens\n    Language: Greek\n    Food: Souvlaki and Feta Cheese\n    Currency: Euro\n    \n\n\n
# Human: What do you know about Germany?'

chain = prompt | chat
chain.invoke({
    "country": "Turkey"
})

### 결과값  
# AI: 
# I know the following:
# Capital: Berlin
# Language: German
# Food: Bratwurst and Sauerkraut
# Currency: Euro


AI: 
I know the following:
Capital: Ankara
Language: Turkish
Food: Kebab and Baklava
Currency: Turkish Lira

AIMessageChunk(content='AI: \nI know the following:\nCapital: Ankara\nLanguage: Turkish\nFood: Kebab and Baklava\nCurrency: Turkish Lira')